# Importing the required libraries

In [5]:
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None 

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

In [6]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bhavana\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Bhavana\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [7]:
from bs4 import BeautifulSoup
from tqdm import tqdm

tqdm.pandas()

In [8]:
from collections import Counter
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
from pprint import pprint
from gensim.models.coherencemodel import CoherenceModel

In [9]:
import os
from gensim.models.wrappers import LdaMallet
os.environ['MALLET_HOME'] = 'C:\\Users\\BOBBY\\Downloads\\mallet-2.0.8'
mallet_path = 'C:\\Users\\BOBBY\\Downloads\\mallet-2.0.8\\bin\\mallet'

# Importing the datasets

In [10]:
#The whole dataset is in the data folder in 5 different files
df1 = pd.read_csv('data/original_data1.csv')
df2 = pd.read_csv('data/original_data2.csv')
df3 = pd.read_csv('data/original_data3.csv')
df4 = pd.read_csv('data/original_data4.csv')
df5 = pd.read_csv('data/original_data5.csv')

In [11]:
#All the different data from differnt datasets is concatenated to form a single dataset
df = pd.concat([df1,df2,df3,df4,df5], axis=0)

In [12]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [13]:
df.reset_index(drop=True, inplace=True)

# Understanding the dataset

For this model, we want to label the answers, so we will be needing only the 'id', 'tags' and 'answers' columns. Other columns are ignored. The required columns are copied into another dataframe, so the original dataframe can be used when required. 

In [14]:
data = df[['id','tags','answers']]

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       100000 non-null  int64 
 1   tags     100000 non-null  object
 2   answers  100000 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.3+ MB


In [16]:
data.isnull().sum()

id         0
tags       0
answers    0
dtype: int64

In [17]:
min_ans_len = data['answers'].str.len().min()
max_ans_len = data['answers'].str.len().max()

In [18]:
data[data['answers'].str.len()==min_ans_len]

,id,tags,answers
43053,4350260,python|inheritance|decorator,"<p>No, it doesn't.</p>"


In [19]:
data[data['answers'].str.len()==max_ans_len]

,id,tags,answers
87503,64587303,python|numpy|image-processing|convolution,<p>I've implemented several very fast solution...


# Text Processing

In [20]:
data['tags'] = data['tags'].str.replace('|',' ')
data['tags'] = data['tags'].str.replace('python-3.x','python')
data['tags'] = data['tags'].str.replace('python-2.7','python')

<ipython-input-20-b360dca21188>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  data['tags'] = data['tags'].str.replace('|',' ')
<ipython-input-20-b360dca21188>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['tags'] = data['tags'].str.replace('python-3.x','python')
<ipython-input-20-b360dca21188>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['tags'] = data['tags'].str.replace('python-2.7','python')


In [21]:
#def parser(text):
#    soup = BeautifulSoup(text, 'html.parser')
#    while(soup.code):
#        soup.code.decompose()
#    return soup.text

In [22]:
#removing html tags
data['answers'] = data['answers'].apply(lambda text: BeautifulSoup(text, "lxml").text)

In [23]:
#converting all letters to lowercase
data['answers'] = data['answers'].str.lower()
data['tags'] = data['tags'].str.lower()

In [24]:
#removing all punctuations
data['answers'] = data['answers'].str.replace('[^\w\s]',' ')
data['tags'] = data['tags'].str.replace('[^\w\s]',' ')

<ipython-input-24-bd614c22202c>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['answers'] = data['answers'].str.replace('[^\w\s]',' ')
<ipython-input-24-bd614c22202c>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['tags'] = data['tags'].str.replace('[^\w\s]',' ')


In [25]:
stem_word_list = [('np ', 'numpy '), ('plt', 'plot'), ('correctly', 'correct'), ('containing', 'contain'), ('better', 'good'),
                  ('best', 'good'), ('strings', 'string'), ('arrays', 'array'), ('variables', 'variable'), ('calls', 'call'), 
                  ('called', 'call'), ('calling', 'call'), ('returns', 'return'), ('results', 'result'), ('values', 'value'), 
                  ('val ', 'value '), ('images', 'image'), ('img', 'image'), ('png', 'image'), ('tuples', 'tuple'), ('arguments', 'argument'), 
                  ('args', 'argument'), ('argv', 'argument'), ('parameters', 'parameter'), ('params', 'parameter'), ('param', 'parameter'), 
                  ('prints', 'print'), ('lists', 'list'), ('runs', 'run'), ('models', 'model'), ('headers', 'header'), ('installation', 'install'), 
                  ('installed','install'), ('tf','tensorflow'), ('found', 'find'), ('lines', 'line'), ('pd', 'pandas'), 
                  ('df', 'dataframe'), ('lib ', 'library '), ('https', 'http'), ('works', 'work')]

In [26]:
for (word, repl_word) in stem_word_list:
    data['answers'] = data['answers'].str.replace(word, repl_word)

In [27]:
#tokenizing text
data['ans_tokenized'] = data['answers'].apply(lambda text: [word for word in nltk.word_tokenize(text) if word.isalpha()])

In [28]:
for i in range(0,data.shape[0]):
    data['tags'][i] = data['tags'][i].split(' ')

In [29]:
#data['ans_tokenized'] = data['ans_tokenized'] + data['tags']

In [30]:
#lemmatizer = WordNetLemmatizer()
#def lemmatize_text(text):
#    return [lemmatizer.lemmatize(word) for word in text]

In [31]:
#data['ans_tokenized'].apply(lemmatize_text)

In [32]:
#function for filtering stop words
def filter_stop_words(words):
    words_filtered = []
    for word in words:
        if word not in stop_words:
            words_filtered.append(word)
    return words_filtered

In [33]:
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

data['ans_tokenized'] = data['ans_tokenized'].apply(filter_stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bhavana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
data

,id,tags,answers,ans_tokenized
0,62766758,"[python, pandas, shell, numpy, google, cloud, ...",container optimized os cos has several limit...,"[container, optimized, os, cos, several, limit..."
1,62766758,"[python, pandas, shell, numpy, google, cloud, ...",i m pretty sure that in case of python librari...,"[pretty, sure, case, python, libraries, native..."
2,62742938,"[python, modulenotfounderror]",the standard library has a module runpy for th...,"[standard, library, module, runpy, purpose, ma..."
3,62741826,"[python, pandas, performance, data, science]",hello and welcome to stackoverflow \nin pandas...,"[hello, welcome, stackoverflow, pandas, rule, ..."
4,62741826,"[python, pandas, performance, data, science]",i guess you want to groupby and exclude all th...,"[guess, want, groupby, exclude, elements, appe..."
...,...,...,...,...
99995,35898687,"[python, apache, spark, pyspark, apache, spark...",just for fun non udataframe solution \nfrom py...,"[fun, non, udataframe, solution, pyspark, sql,..."
99996,35898687,"[python, apache, spark, pyspark, apache, spark...",the following code does the requested task an...,"[following, code, requested, task, user, defin..."
99997,35937882,"[python, python, user, interface, tkinter]",at the end of your code just add window mainl...,"[end, code, add, window, mainloop, tells, tkin..."
99998,35937882,"[python, python, user, interface, tkinter]",you didn t do a \nwindow mainloop \n\nat the...,"[window, mainloop, end, check, tkinter, docs, ..."


In [35]:
count = Counter()
def count_tag(answer):
    for word in answer:
        count[word] += 1

In [36]:
data['ans_tokenized'].apply(count_tag)
len(count.values())

111579

In [37]:
word_list = []
for word, cou in count.items():
    if(cou>1000):
        word_list.append(word)

Some of the words are not useful, which are therefore added to the list of 'stop_words' and again the answer text is filtered.

In [38]:
stop_words.update(['several', 'could', 'many', 'may', 'non', 'quite', 'v', 'l', 'e', 'g', 'rw', 'r', 'xr', 'x', 'rwxr', 'think',
                   'sane', 'hello', 'welcome', 'still', 'normal', 'much', 'c', 'p', 'n', 'k', 'b', 'h', 'lot', 'us', 'kind', 
                   'q', 'w', 'z', 'en', 'ax', 'j', 'python', 'code', 'using', 'need', 'get', 'py', 'u', 'keep', 'general', 
                   'also', 'really', 'normal', 'able', 'maybe', 'things', 'everything', 'happens', 'actually', 'lst', 'already',
                  'within', 'li', 'python', 'based', 'user', 'np', 'pd', 'tf', 'id', 'work', 'write', 'df', 'np', 'pd', 'make',
                  'said'])

In [39]:
data['ans_tokenized'] = data['ans_tokenized'].apply(filter_stop_words)

In [40]:
answers = data['ans_tokenized']

In [41]:
answers

0        [container, optimized, os, cos, limitations, r...
1        [pretty, sure, case, libraries, natives, messa...
2        [standard, library, module, runpy, purpose, sc...
3        [stackoverflow, pandas, rule, thumb, raw, loop...
4        [guess, want, groupby, exclude, elements, appe...
                               ...                        
99995    [fun, udataframe, solution, pyspark, sql, func...
99996    [following, requested, task, defined, function...
99997    [end, add, window, mainloop, tells, tkinter, f...
99998    [window, mainloop, end, check, tkinter, docs, ...
99999    [whenever, faced, complex, xml, consider, xslt...
Name: ans_tokenized, Length: 100000, dtype: object

# Building the model

In [42]:
#Create dictionary and corpus
id2word = corpora.Dictionary(answers)
corpus = [id2word.doc2bow(answer) for answer in answers]

In [ ]:
#Building an lda model
ldamodel = LdaMulticore(corpus=corpus, num_topics=8, id2word=id2word, passes=5)

In [ ]:
pprint(ldamodel.show_topics(formatted=False))

In [ ]:
ldamodel.log_perplexity(corpus)

In [ ]:
coherence_lda = CoherenceModel(model=ldamodel, texts=answers, dictionary=id2word, coherence='c_v')
score = coherence_lda.get_coherence()

In [ ]:
print("Coherence score of lda model: ",score)

In [43]:
ldamallet_model = LdaMallet(mallet_path, corpus=corpus, num_topics=8, id2word=id2word)

CalledProcessError: Command 'C:\Users\BOBBY\Downloads\mallet-2.0.8\bin\mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\Users\Bhavana\AppData\Local\Temp\f52527_corpus.txt --output C:\Users\Bhavana\AppData\Local\Temp\f52527_corpus.mallet' returned non-zero exit status 1.

In [ ]:
pprint(ldamallet_model.show_topics(formatted=False))

In [66]:
coherence_ldamallet = CoherenceModel(model=ldamallet_model, texts=answers, dictionary=id2word, coherence='c_v')
score = coherence_ldamallet.get_coherence()

In [67]:
print("Coherence score of lda mallet model: ",score)

Coherence score of lda mallet model:  0.5865867956632709


By comparing the scores, it is clearly evident that LDA Mallet model works better that LDA multicore model. We can use LDA mallet model and find optimal number of topics.

# Finding optimal number of topics

In [ ]:
def coh_value(id2word, corpus, answers, start, stop, step):
    coh_values_list = []
    model_list = []
    for num_topics in range(start, stop, step):
        model = LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=answers, dictionary=id2word, coherence='c_v')
        coh_values_list.append(coherencemodel.get_coherence())
    return model_list, coh_values_list

In [ ]:
start = 2
stop = 20
step = 2

In [311]:
model_list, coh_values_list = coh_value(id2word, corpus, answers, start, stop, step)

2
4
6
8
10
12
14
16
18


In [ ]:
topic_num = range(start, stop, step)
plt.plot(topic_num, coh_values_list)
plt.title("Coherence value score for the number of topics")
plt.xlabel("No. of Topics")
plt.ylabel("Coherence score")
#plt.legend(("coherence_values"), loc='best')
plt.show()

In [315]:
for i in range(start, stop, step):
    print("No. of topics:  ", i, ", Coherence Value: ", coh_values_list[(i-start)//2])

No. of topics:   2 , Coherence Value:  0.4095438810466185
No. of topics:   4 , Coherence Value:  0.5342842390038446
No. of topics:   6 , Coherence Value:  0.5682253615521279
No. of topics:   8 , Coherence Value:  0.5837369851009455
No. of topics:   10 , Coherence Value:  0.6068811678520329
No. of topics:   12 , Coherence Value:  0.6149607929334713
No. of topics:   14 , Coherence Value:  0.6138146710755172
No. of topics:   16 , Coherence Value:  0.6174509538255538
No. of topics:   18 , Coherence Value:  0.6161918846495307


In [68]:
final_model = final_model[3]
model_topics = final_model.show_topics(formatted=False)
pprint(final_model.print_topics(num_words=10))

[(0,
  '0.081*"dataframe" + 0.036*"pandas" + 0.032*"true" + 0.028*"index" + '
  '0.028*"key" + 0.027*"false" + 0.021*"row" + 0.019*"nan" + 0.016*"column" + '
  '0.016*"columns"'),
 (1,
  '0.062*"list" + 0.048*"print" + 0.027*"result" + 0.022*"return" + '
  '0.020*"number" + 0.020*"int" + 0.020*"append" + 0.019*"len" + 0.018*"range" '
  '+ 0.017*"item"'),
 (2,
  '0.052*"line" + 0.043*"file" + 0.038*"string" + 0.025*"print" + 0.025*"data" '
  '+ 0.023*"open" + 0.019*"import" + 0.018*"split" + 0.016*"read" + '
  '0.015*"output"'),
 (3,
  '0.024*"path" + 0.020*"install" + 0.018*"os" + 0.016*"run" + 0.013*"version" '
  '+ 0.012*"library" + 0.011*"module" + 0.010*"problem" + 0.009*"command" + '
  '0.009*"script"'),
 (4,
  '0.055*"numpy" + 0.028*"array" + 0.023*"image" + 0.022*"import" + '
  '0.021*"plot" + 0.014*"data" + 0.014*"random" + 0.010*"size" + '
  '0.010*"tensorflow" + 0.008*"shape"'),
 (5,
  '0.032*"http" + 0.018*"import" + 0.016*"html" + 0.014*"request" + '
  '0.013*"url" + 0.012*

# Creating answer dataset with topics

In [69]:
topics = final_model[corpus]

In [70]:
topics

[[(0, 0.039808917197452234),
  (1, 0.04617834394904459),
  (2, 0.05325548478414721),
  (3, 0.5727176220806796),
  (4, 0.09005661712668084),
  (5, 0.06245576786978062),
  (6, 0.06245576786978062),
  (7, 0.07307147912243456)],
 [(0, 0.0791343669250646),
  (1, 0.09593023255813954),
  (2, 0.11918604651162791),
  (3, 0.2638888888888889),
  (4, 0.10368217054263566),
  (5, 0.07525839793281654),
  (6, 0.1101421188630491),
  (7, 0.1527777777777778)],
 [(0, 0.10397897897897898),
  (1, 0.09797297297297297),
  (2, 0.11148648648648649),
  (3, 0.2766516516516517),
  (4, 0.09346846846846847),
  (5, 0.09496996996996997),
  (6, 0.10998498498498499),
  (7, 0.11148648648648649)],
 [(0, 0.2631448412698413),
  (1, 0.15203373015873015),
  (2, 0.0716765873015873),
  (3, 0.06969246031746032),
  (4, 0.08754960317460317),
  (5, 0.06572420634920635),
  (6, 0.1827876984126984),
  (7, 0.10739087301587301)],
 [(0, 0.24060457516339873),
  (1, 0.1491013071895425),
  (2, 0.09191176470588237),
  (3, 0.09517973856209151

In [71]:
topic_df = pd.DataFrame()

In [72]:
for i, topic_vector in enumerate(topics):
    topic_list = []
    topic_vector = sorted(topic_vector, key = lambda topic: (topic[1]), reverse=True)
    for j, (topic_num, prob) in enumerate(topic_vector):
        prob = round(prob,2)*100
        if(prob > 14):
            topic = "Topic " + str(j)
            topic_list.append(topic)
        else:
            break
    topics_col = ", ".join([topic for topic in topic_list])
    if(i%1000==0): print(i)
    topic_df = topic_df.append(pd.Series([df.iloc[i]['id'], df.iloc[i]['answers'], topics_col]), ignore_index=True) 

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000


In [73]:
topic_df

,0,1,2
0,62766758.0,<p>Container-Optimized OS (COS) has several li...,Topic 0
1,62766758.0,<p>I'm pretty sure that in case of Python libr...,"Topic 0, Topic 1"
2,62742938.0,"<p>The standard library has a module <a href=""...",Topic 0
3,62741826.0,<p>Hello and welcome to StackOverflow.</p>\r\n...,"Topic 0, Topic 1, Topic 2"
4,62741826.0,<p>I guess you want to <code>groupby</code> an...,"Topic 0, Topic 1"
...,...,...,...
99995,35898687.0,<p>Just for fun non-UDF solution:</p>\r\n\r\n<...,"Topic 0, Topic 1"
99996,35898687.0,<p>The following code does the requested task....,"Topic 0, Topic 1, Topic 2"
99997,35937882.0,"<p>At the end of your code, just add <code>win...",Topic 0
99998,35937882.0,<p>You didn't do a </p>\r\n\r\n<pre><code>wind...,"Topic 0, Topic 1"


In [74]:
topic_df.columns = ['id', 'answer', 'topic list']

In [75]:
topic_df.to_csv('data/keyword_answer.csv', index=False)

# Creating dataset for topics and its keywords

In [76]:
keyword_df = pd.DataFrame()

In [77]:
for topic_num in range(0,8):
    topic_keyword_list = final_model.show_topic(topic_num)
    keyword_list = []
    for word in topic_keyword_list:
        keyword_list.append(word[0])
    keyword_df = keyword_df.append(pd.Series(["Topic "+str(topic_num), keyword_list]), ignore_index=True)

In [78]:
keyword_df.columns = ['Topic', 'Keywords']

In [79]:
keyword_df

,Topic,Keywords
0,Topic 0,"[dataframe, pandas, true, index, key, false, r..."
1,Topic 1,"[list, print, result, return, number, int, app..."
2,Topic 2,"[line, file, string, print, data, open, import..."
3,Topic 3,"[path, install, os, run, version, library, mod..."
4,Topic 4,"[numpy, array, image, import, plot, data, rand..."
5,Topic 5,"[http, import, html, request, url, json, djang..."
6,Topic 6,"[return, def, function, class, call, object, a..."
7,Topic 7,"[time, def, import, text, start, root, process..."


In [80]:
keyword_df.to_csv('data/keywords.csv', index=False)